# Haiku aus dem Nationalrat

Der korrekte Plural von Haiku ist Haiku.  
Der Verständlichkeit halber wird im Code aber *Haikus* verwendet.

protokolle laden und flatten

In [15]:
import json
import pandas as pd

with open("woswormeileistung/data/sessions.json") as f:
    sessions = json.load(f)

wortmeldungen = pd.json_normalize(
    sessions,
    record_path=["sections"],
    meta=["period", "sessionNumber", "date"],
)

wortmeldungen = wortmeldungen[["period", "sessionNumber", "date", "speaker", "text"]]

wortmeldungen.head()

,period,sessionNumber,date,speaker,text
0,XXVII,276,2024-09-18T00:00:00,88386,Präsident Mag. Wolfgang Sobotka: Meine sehr ge...
1,XXVII,276,2024-09-18T00:00:00,88386,Präsident Mag. Wolfgang Sobotka: Meine sehr ge...
2,XXVII,276,2024-09-18T00:00:00,88386,Präsident Mag. Wolfgang Sobotka: Der Herr Bund...
3,XXVII,276,2024-09-18T00:00:00,88386,Präsident Mag. Wolfgang Sobotka: Die Amtlichen...
4,XXVII,276,2024-09-18T00:00:00,88386,Präsident Mag. Wolfgang Sobotka: Ich darf beka...


sprecher:in label aus text entfernen

In [16]:
wortmeldungen["text"] = wortmeldungen["text"].str.split(": ", n=1).str[1]
wortmeldungen.head()

,period,sessionNumber,date,speaker,text
0,XXVII,276,2024-09-18T00:00:00,88386,Meine sehr geehrten Damen und Herren Abgeordne...
1,XXVII,276,2024-09-18T00:00:00,88386,Meine sehr geehrten Damen und Herren auf der G...
2,XXVII,276,2024-09-18T00:00:00,88386,Der Herr Bundespräsident hat mit Entschließung...
3,XXVII,276,2024-09-18T00:00:00,88386,Die Amtlichen Protokolle der 272. und der 273....
4,XXVII,276,2024-09-18T00:00:00,88386,"Ich darf bekannt geben, dass von der Bundeswah..."


dev mode: probelauf auf 1000 texte

In [17]:
wortmeldungen = wortmeldungen.sample(1000)

haiku finden

In [18]:
import spacy
import pyphen
import re
from tqdm import tqdm

#!python -m spacy download de_core_news_lg
!python -m spacy download de_core_news_sm

#nlp = spacy.load("de_core_news_lg")
nlp = spacy.load("de_core_news_sm")
dic = pyphen.Pyphen(lang="de_DE")

def extract_haiku(sentence):
    s = sentence.strip()
    s = s.replace(",", "").replace(".", "").replace("!", "").replace("?", "").replace("-", "")
    if not re.fullmatch(r"[A-Za-z ]+", s):
        return None
    
    words = s.split()
    syllable_counts = [dic.inserted(w).count("-") + 1 for w in words]

    line_limits = [5, 7, 5]
    line_idx = 0
    line_sum = 0
    extracted_haiku = [[], [], []]

    for word, syllable_count in zip(words, syllable_counts):
        if line_idx >= 3 or line_sum > line_limits[line_idx]:
            break

        if line_sum < line_limits[line_idx]:
            extracted_haiku[line_idx].append(word)
            line_sum += syllable_count
            if line_sum == line_limits[line_idx]:
                line_idx += 1
                line_sum = 0

    haiku_word_count = sum(len(line) for line in extracted_haiku)
    if line_idx == 3 and line_sum == 0 and haiku_word_count == len(words):
        return [" ".join(line) for line in extracted_haiku]
    
    return None


haikus = []
context_indices = []
for doc in tqdm(nlp.pipe(wortmeldungen["text"].fillna("").astype(str), batch_size=50), total=len(wortmeldungen)):
    haikus.append([extract_haiku(sentence.text) for sentence in doc.sents])
    context_indices.append([(sentence.start_char, sentence.end_char) for sentence in doc.sents])
    
wortmeldungen["lines"] = haikus
wortmeldungen["contextIndices"] = context_indices
wortmeldungen.head()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 7.5 MB/s  0:00:02 eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


100%|██████████| 1000/1000 [00:45<00:00, 21.76it/s]


,period,sessionNumber,date,speaker,text,lines,contextIndices
87390,XXV,27,2014-05-21T00:00:00,35521,Der eingebrachte Antrag ist ausreichend unters...,"[None, None, None, None, None, None, None, Non...","[(0, 88), (88, 171), (172, 575), (575, 688), (..."
136624,XXIV,14,2009-02-26T00:00:00,2834,Wir gelangen nunmehr zu den Punkten 14 bis 16 ...,"[None, None, None, None, None, None]","[(0, 119), (119, 174), (174, 224), (225, 256),..."
30784,XXVII,91,2021-03-25T00:00:00,88386,Zu Wort gemeldet ist Abgeordneter Wurm. – Bitt...,"[None, None, None]","[(0, 39), (40, 49), (49, 57)]"
120953,XXIV,77,2010-09-22T00:00:00,3604,Herr Abgeordneter Dr. Kurt Grünewald hat sich ...,"[None, None, None]","[(0, 99), (100, 123), (123, 131)]"
67146,XXV,138,2016-07-07T00:00:00,83124,Herr Präsident! Geschätzter Herr Minister! Bev...,"[None, None, None, None, None, None, None, Non...","[(0, 15), (16, 42), (43, 214), (215, 235), (23..."


liste auf haikus flatten und contextBefore/after ableiten

In [19]:
haikus = wortmeldungen.explode(["lines", "contextIndices"]).dropna(subset=["lines"])
haikus["contextBefore"] = haikus.apply(lambda row: row["text"][: row["contextIndices"][0]], axis=1)
haikus["contextAfter"] = haikus.apply(lambda row: row["text"][row["contextIndices"][1] :], axis=1)
haikus = haikus.drop(columns=["text", "contextIndices"])
haikus.head()

,period,sessionNumber,date,speaker,lines,contextBefore,contextAfter
114829,XXIV,103,2011-04-29T00:00:00,40628,"[Es geht da nicht um, Mitwirkungspflicht sonde...",Sehr geehrte Damen und Herren! Dieses Paket gi...,(Beifall bei den Grünen.)\nDer Zynismus diese...
178577,XXII,41,2003-12-04T00:00:00,2854,"[Nein nein darum geht, es nicht die Strukturfr...",Sehr geehrter Herr Bundesminister! Wenn man Ih...,Dann – „Zufall“ – kommt in letzter Minute – Mu...
171920,XXII,79,2004-10-14T00:00:00,14696,"[Es muss ein langer, Aufschwung sein der so la...",Frau Präsidentin! Herr Staatssekretär! Geschät...,"Dann hat sich Herr Mag. Grasser gedacht, dass ..."
44601,XXVI,80,2019-06-12T00:00:00,2889,"[Kollege Stefan, was haben wir zu diesem, Them...",Frau Präsidentin! Herr Vizekanzler! Kolleginne...,"Jahrelang war die FPÖ die Partei, die genau d..."
41674,XXVII,10,2020-01-22T00:00:00,88386,"[Zu Wort gemeldet, ist Herr Abgeordneter, Klub...",,– Bitte.\n11.47.04


gesetzgebungsperiode als arabische zahl für tooltip

In [20]:
import roman

haikus["periodRoman"] = haikus["period"]
haikus["period"] = haikus["period"].apply(lambda periodRoman: roman.fromRoman(periodRoman))
haikus.head()

,period,sessionNumber,date,speaker,lines,contextBefore,contextAfter,periodRoman
114829,24,103,2011-04-29T00:00:00,40628,"[Es geht da nicht um, Mitwirkungspflicht sonde...",Sehr geehrte Damen und Herren! Dieses Paket gi...,(Beifall bei den Grünen.)\nDer Zynismus diese...,XXIV
178577,22,41,2003-12-04T00:00:00,2854,"[Nein nein darum geht, es nicht die Strukturfr...",Sehr geehrter Herr Bundesminister! Wenn man Ih...,Dann – „Zufall“ – kommt in letzter Minute – Mu...,XXII
171920,22,79,2004-10-14T00:00:00,14696,"[Es muss ein langer, Aufschwung sein der so la...",Frau Präsidentin! Herr Staatssekretär! Geschät...,"Dann hat sich Herr Mag. Grasser gedacht, dass ...",XXII
44601,26,80,2019-06-12T00:00:00,2889,"[Kollege Stefan, was haben wir zu diesem, Them...",Frau Präsidentin! Herr Vizekanzler! Kolleginne...,"Jahrelang war die FPÖ die Partei, die genau d...",XXVI
41674,27,10,2020-01-22T00:00:00,88386,"[Zu Wort gemeldet, ist Herr Abgeordneter, Klub...",,– Bitte.\n11.47.04,XXVII


personen verknüpfen

In [21]:
personen = pd.read_json("woswormeileistung/data/persons.json")
personen = personen[["id", "name", "parties", "imageUrl"]]
personen["id"] = personen["id"].astype(str)
haikus = haikus.merge(personen, left_on="speaker", right_on="id", how="left")
haikus = haikus.drop(columns=["id", "speaker"])
haikus.head()

,period,sessionNumber,date,lines,contextBefore,contextAfter,periodRoman,name,parties,imageUrl
0,24,103,2011-04-29T00:00:00,"[Es geht da nicht um, Mitwirkungspflicht sonde...",Sehr geehrte Damen und Herren! Dieses Paket gi...,(Beifall bei den Grünen.)\nDer Zynismus diese...,XXIV,Mag. Albert Steinhauser,[GRÜNE],https://parlament.gv.at/dokument/bild/45823/45...
1,22,41,2003-12-04T00:00:00,"[Nein nein darum geht, es nicht die Strukturfr...",Sehr geehrter Herr Bundesminister! Wenn man Ih...,Dann – „Zufall“ – kommt in letzter Minute – Mu...,XXII,Karl Öllinger,[GRÜNE],https://parlament.gv.at/dokument/bild/63474/63...
2,22,79,2004-10-14T00:00:00,"[Es muss ein langer, Aufschwung sein der so la...",Frau Präsidentin! Herr Staatssekretär! Geschät...,"Dann hat sich Herr Mag. Grasser gedacht, dass ...",XXII,Heidemarie Rest-Hinterseer,[GRÜNE],https://parlament.gv.at/dokument/bild/21017/21...
3,26,80,2019-06-12T00:00:00,"[Kollege Stefan, was haben wir zu diesem, Them...",Frau Präsidentin! Herr Vizekanzler! Kolleginne...,"Jahrelang war die FPÖ die Partei, die genau d...",XXVI,Dr. Johannes Jarolim,[SPÖ],https://parlament.gv.at/dokument/bild/71826/71...
4,27,10,2020-01-22T00:00:00,"[Zu Wort gemeldet, ist Herr Abgeordneter, Klub...",,– Bitte.\n11.47.04,XXVII,Mag. Wolfgang Sobotka,[ÖVP],https://parlament.gv.at/dokument/bild/200697/2...


exportieren

In [22]:
haikus.to_json("web/haiku.json", orient="records", force_ascii=False, indent=2)